In [110]:
import numpy as np
import os.path

In [111]:
class patchFamily2D:
    def __init__(self,file_ID):
        #
        self.U = [] # Knot Vectors
        self.CP = [] # Kontrol Points
        self.p = [] # Orders
        self.nsf = [] # Number of Shape Functions
        # Connectivity
        self.nel = []
        self.nnp = []
        self.nen = []
        self.INC = []
        self.IEN = []
        # Read Patch Data
        file_exists = os.path.exists(file_ID+'1')
        fNo = 1
        while (file_exists == True):
            f = open(file_ID+str(fNo), "r")
            data = f.readlines()
            # Knot Vector - V
            lenV = int(data[0])
            V_p = np.zeros(lenV)
            dataV = data[1].split(" ")
            for i in range(0,lenV):
                V_p[i] = float(dataV[i])
            #self.V.append(V_p)
            # Knot Vector - U
            lenU = int(data[2])
            U_p = np.zeros(lenU)
            dataU = data[3].split(" ")
            for i in range(0,lenU):
                U_p[i] = float(dataU[i])
            self.U.append([])
            self.U[fNo-1].append(U_p)
            self.U[fNo-1].append(V_p)
            # Control Points:
            lenCP = int(data[4])
            CP_p = np.zeros((lenCP,4))
            for i in range(0,lenCP):
                dataCP = data[i+5].split(" ")
                for j in range(0,4):
                    CP_p[i][j] = float(dataCP[j])
            self.CP.append(CP_p)
            # Order:
            p1 = 0
            p2 = 0
            while True:
                if (U_p[p1] == U_p[p1+1]):
                    p1 += 1
                else:
                    break
            while True:
                if (V_p[p2] == V_p[p2+1]):
                    p2 += 1
                else:
                    break  
            self.p.append(np.array([p1,p2]))
            n = len(U_p)-p1-1
            m = len(V_p)-p2-1
            self.nsf.append(np.array([n,m]))
            # Connectivity:
            nel = (n-p1)*(m-p2)
            self.nel.append(nel)
            nnp = n*m
            self.nnp.append(nnp)
            nen = (p1+1)*(p2+1)
            self.nen.append(nen)
            #
            INC = np.zeros((n*m,2)).astype(int)
            IEN = np.zeros((nel,nen)).astype(int)
            A = 0
            B = 0
            for j in range(1,m+1):
                for i in range(1,n+1):
                    INC[A][0] = (i-1)
                    INC[A][1] = (j-1)
                    if (i >= p1+1 and j >= p2+1):
                        for k in range(0,p2+1):
                            for l in range(0,p1+1):
                                C = int(k*(p1+1)+l)
                                IEN[B][C] = ((A-k*n-l))
                        B += 1
                    A += 1
            self.INC.append(INC)
            self.IEN.append(IEN)
            #
            fNo = fNo + 1
            file_exists = os.path.exists(file_ID+str(fNo))
        # Number of Patches:
        self.np = fNo-1
        
    def findSpan(self,pN,pD,u):
        if (u == self.U[pN][pD][self.nsf[pN][pD]+1]):
            return n
        low = self.p[pN][pD]
        high = self.nsf[pN][pD]+1
        mid = int((low+high)/2)
        while (u<self.U[pN][pD][mid] or u>self.U[pN][pD][mid+1]):
            if (u<self.U[pN][pD][mid]):
                high = mid
            else:
                low = mid
            mid = int(np.floor((low+high)/2))
        return mid
    
    def dersBasisFuns(self,pN,pD,u,n):
        i = self.findSpan(pN,pD,u)
        left = np.zeros(self.p[pN][pD]+1)
        right = np.zeros(self.p[pN][pD]+1)
        ndu = np.zeros((self.p[pN][pD]+1,self.p[pN][pD]+1))
        a = np.zeros((2,self.p[pN][pD]+1))
        ders = np.zeros((n+1,self.p[pN][pD]+1))
        #
        ndu[0][0] = 1.0
        for j in range(1,self.p[pN][pD]+1):
            left[j] = u-self.U[pN][pD][i+1-j]
            right[j] = self.U[pN][pD][i+j]-u
            saved = 0.0
            for r in range(0,j):
                ndu[j][r] = right[r+1]+left[j-r]
                temp = ndu[r][j-1]/ndu[j][r]
                ndu[r][j] = saved+right[r+1]*temp
                saved = left[j-r]*temp
            ndu[j][j] = saved
        for j in range(0,self.p[pN][pD]+1):
            ders[0][j] = ndu[j][self.p[pN][pD]]
        for r in range(0,self.p[pN][pD]+1):
            s1 = 0
            s2 = 1
            a[0][0] = 1.0
            for k in range(1,n+1):
                d =0.0
                rk = r-k
                pk = self.p[pN][pD]-k
                if (r >= k):
                    a[s2][0] = a[s1][0]/ndu[pk+1][rk]
                    d = a[s2][0]*ndu[rk][pk]
                if (rk >= -1):
                    j1 = 1
                else:
                    j1 = -rk
                if (r-1 <= pk):
                    j2 = k-1
                else:
                    j2 = self.p[pN][pD]-r
                for j in range(j1,j2+1):
                    a[s2][j] = (a[s1][j]-a[s1][j-1])/ndu[pk+1][rk+j]
                    d += a[s2][j]*ndu[rk+j][pk]
                if (r <= pk):
                    a[s2][k] = -a[s1][k-1]/ndu[pk+1][r]
                    d += a[s2][k]*ndu[r][pk]
                ders[k][r] = d
                j = s1
                s1 = s2
                s2 = j
        r = self.p[pN][pD]
        for k in range(1,n+1):
            for j in range(0,self.p[pN][pD]+1):
                ders[k][j] *= r
            r *= (self.p[pN][pD]-k)
        return ders
    
    def getCP(self,pN,p_el):
        cp = np.zeros((self.nen[pN],4))
        iu = self.INC[pN][self.IEN[pN][p_el][0],0]
        iv = self.INC[pN][self.IEN[pN][p_el][0],1]
        ny = iv-self.p[pN][1]
        k = 0
        for j in range(0,self.p[pN][1]+1):
            nx = iu-self.p[pN][0]
            for i in range(0,self.p[pN][0]+1):
                cp[k][:] = self.CP[pN][self.nsf[pN][0]*ny+nx][:]
                nx += 1
                k += 1
            ny += 1
        return cp
    
    def ratBasisFuns2D(self,pN,u,v,du,dv):
        N = self.dersBasisFuns(self,pN,0,u,du)
        M = self.dersBasisFuns(self,pN,1,v,dv)
        R = np.zeros((((self.p[pN][0],self.p[pN][1],du+1,dv+1))))
        wders = np.zeros((du+1,dv+1))
        wMat = np.reshape(CP[:,3],[self.p[pN][0]+1,self.p[pN][1]+1])
        for k in range(0,du+1):
            for l in range(0,dv+1):
                wders[k][l] = N[k][:].dot(wMat.dot(M[l][:]))
                temp1 = (N[k][:].dot(M[l][:]))*wMat
                for j in range(1,l+1):
                    temp1 -= (math.comb(l,j)*wders[0][j])*R[:][:][k][l-j]
                for i in range(1,k+1):
                    temp1 -= (math.comb(k,i)*wders[i][0])*R[:][:][k-i][l]
                    temp2 = np.zeros((p,q))
                    for j in range (1,l+1):
                        temp2 += (math.comb(l,j)*wders[i][j])*R[:][:][k-i][l-j]
                    temp1 -= math.comb(k,i)*temp2
                R[:][:][k][l] = temp1/wders[0][0]
        return R

In [112]:
pf = patchFamily2D('igampm_10x5_')

In [113]:
# Number of Patches
pf.np

1

In [114]:
# Knot Vectors:
pf.U

[[array([0.  , 0.  , 0.  , 0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 ,
         0.45, 0.5 , 0.55, 0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95,
         1.  , 1.  , 1.  ]),
  array([0. , 0. , 0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ,
         1. , 1. ])]]

In [115]:
# Control Points:
pf.CP[0]

array([[ 0.  ,  0.  ,  0.  ,  1.  ],
       [ 0.25,  0.  ,  0.  ,  1.  ],
       [ 0.75,  0.  ,  0.  ,  1.  ],
       ...,
       [ 9.25,  5.  ,  0.  ,  1.  ],
       [ 9.75,  5.  ,  0.  ,  1.  ],
       [10.  ,  5.  ,  0.  ,  1.  ]])

In [116]:
# Order:
pf.p[0]

array([2, 2])

In [117]:
# Number of Shape Functions:
pf.nsf[0]

array([22, 12])

In [118]:
pf.INC[0]

array([[ 0,  0],
       [ 1,  0],
       [ 2,  0],
       [ 3,  0],
       [ 4,  0],
       [ 5,  0],
       [ 6,  0],
       [ 7,  0],
       [ 8,  0],
       [ 9,  0],
       [10,  0],
       [11,  0],
       [12,  0],
       [13,  0],
       [14,  0],
       [15,  0],
       [16,  0],
       [17,  0],
       [18,  0],
       [19,  0],
       [20,  0],
       [21,  0],
       [ 0,  1],
       [ 1,  1],
       [ 2,  1],
       [ 3,  1],
       [ 4,  1],
       [ 5,  1],
       [ 6,  1],
       [ 7,  1],
       [ 8,  1],
       [ 9,  1],
       [10,  1],
       [11,  1],
       [12,  1],
       [13,  1],
       [14,  1],
       [15,  1],
       [16,  1],
       [17,  1],
       [18,  1],
       [19,  1],
       [20,  1],
       [21,  1],
       [ 0,  2],
       [ 1,  2],
       [ 2,  2],
       [ 3,  2],
       [ 4,  2],
       [ 5,  2],
       [ 6,  2],
       [ 7,  2],
       [ 8,  2],
       [ 9,  2],
       [10,  2],
       [11,  2],
       [12,  2],
       [13,  2],
       [14,  2

In [126]:
pf.IEN[0]

array([[ 46,  45,  44, ...,   2,   1,   0],
       [ 47,  46,  45, ...,   3,   2,   1],
       [ 48,  47,  46, ...,   4,   3,   2],
       ...,
       [261, 260, 259, ..., 217, 216, 215],
       [262, 261, 260, ..., 218, 217, 216],
       [263, 262, 261, ..., 219, 218, 217]])

In [123]:
cp = pf.getCP(0,199)
cp

array([[ 9.25,  4.25,  0.  ,  1.  ],
       [ 9.75,  4.25,  0.  ,  1.  ],
       [10.  ,  4.25,  0.  ,  1.  ],
       [ 9.25,  4.75,  0.  ,  1.  ],
       [ 9.75,  4.75,  0.  ,  1.  ],
       [10.  ,  4.75,  0.  ,  1.  ],
       [ 9.25,  5.  ,  0.  ,  1.  ],
       [ 9.75,  5.  ,  0.  ,  1.  ],
       [10.  ,  5.  ,  0.  ,  1.  ]])

In [128]:
wMat = np.reshape(cp[:,3],[pf.p[0][0]+1,pf.p[0][1]+1])
wMat

array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]])

In [134]:
AA = np.array([[1, 2, 3, 4, 5], [3, 5, 2, 6, 7],[1, 7, 3, 9, 5],[2, 4, 5, 6, 7],[1, 3, 7, 8, 4],[3, 5, 6, 2, 8]])
AA

array([[1, 2, 3, 4, 5],
       [3, 5, 2, 6, 7],
       [1, 7, 3, 9, 5],
       [2, 4, 5, 6, 7],
       [1, 3, 7, 8, 4],
       [3, 5, 6, 2, 8]])

In [137]:
(AA[:][3])

array([2, 4, 5, 6, 7])